# Posterior probability

In [my solution](https://www.kaggle.com/somnambwl/santa-2020) of [Santa-2020 competition](https://www.kaggle.com/c/santa-2020/overview), I have used the mode of posterior probability to estimate one-armed bandits' thresholds. However, [Dmitriy Guller](https://www.kaggle.com/dmitriyguller) [suggested](https://www.kaggle.com/c/santa-2020/discussion/216189#1181690) that mean of the posterior distribution may have worked better than mode.

This notebook contains an interactive plot which shows the posterior distribution, it's mode and mean, for comparison. Use the textbox widget below to write "S" for success, "F" for failure or anything else for an unknown result. You can also change the `change_likelihood` function, the default is as for Santa-2020 competition. 

In [ ]:
# Standard library
import copy

# Basic data libraries
import numpy as np

# Interactivity
import ipywidgets as widgets

# Graphs
import matplotlib
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# Set plotly template

import plotly.io as pio

template = pio.templates["simple_white"]

seaborn_colorblind_palette = [(0.00392156862745098, 0.45098039215686275, 0.6980392156862745),
        (0.8705882352941177, 0.5607843137254902, 0.0196078431372549),
        (0.00784313725490196, 0.6196078431372549, 0.45098039215686275),
        (0.8352941176470589, 0.3686274509803922, 0.0),
        (0.8, 0.47058823529411764, 0.7372549019607844),
        (0.792156862745098, 0.5686274509803921, 0.3803921568627451),
        (0.984313725490196, 0.6862745098039216, 0.8941176470588236),
        (0.5803921568627451, 0.5803921568627451, 0.5803921568627451),
        (0.9254901960784314, 0.8823529411764706, 0.2),
        (0.33725490196078434, 0.7058823529411765, 0.9137254901960784)]

template.layout["colorway"] = [matplotlib.colors.to_hex(color) for color in seaborn_colorblind_palette]
template.layout["xaxis"]["showgrid"] = True
template.layout["yaxis"]["showgrid"] = True
template.layout["xaxis"]["mirror"] = True
template.layout["yaxis"]["mirror"] = True
template.layout["font"]["size"] = 16
template.layout["annotationdefaults"]["font"]["size"] = 32

pio.templates.default = template

In [ ]:
def change_likelihood(initial_likelihood, N):
    """
    Computes likelihood of gaining a reward after `N`-th pull.
    
    Parameters
    ==========
    initial_likelihood: np.array
        Possible initial likelihood to gain a reward.
    N: int
        Number of pulls
        
    Returns
    =======
    np.array
        Likelihood to gain a reward after `N`-th pull.
    """
    return initial_likelihood * 0.97**N

In [ ]:
def apply_history_results(prior, history_results):
    """
    Update prior probability using results from history.
    
    Parameters
    ==========
    prior: np.array
        Prior probability
    history_resutls: list of str
        "S" for success, "F" for failure, everything else
        for an unknown result
    
    Returns
    =======
    np.array
        Posterior probability
    """
    posterior = prior
    initial_likelihood = np.arange(0, 101) / 101
    likelihood = copy.deepcopy(initial_likelihood)
    for N_pulls, outcome in enumerate(history_results, 1):
        if outcome == "S":
            posterior *= likelihood
            posterior /= posterior.sum()
            likelihood = change_likelihood(initial_likelihood, N_pulls)
        elif outcome == "F":
            posterior *= (1-likelihood)
            posterior /= posterior.sum()
            likelihood = change_likelihood(initial_likelihood, N_pulls)
        else:
            likelihood = change_likelihood(initial_likelihood, N_pulls)
    return posterior

In [ ]:
hypotheses = np.arange(0, 101)
prior = np.ones(101) / 101 
history_results = []
posterior = apply_history_results(prior, history_results)
mode = posterior.argmax()
mean = np.average(np.arange(0, 101), weights=posterior)
yrange = (np.ceil(posterior.max() * 100)+1) / 100

In [ ]:
# Create Plotly interactive graph
history = widgets.Text(
    value="",
    description="Results:",)

bars = go.Bar(x=hypotheses, y=posterior,
              name="Posterior probability",
              hovertemplate=
                "Initial likelihood of reward: %{x}<br>" +
                "Posterior probability: %{y:0.5f}<br><extra></extra>",)
mode_line = go.Scatter(x=[mode]*100, y=np.linspace(0, yrange, 100),
                       name="Mode", 
                       hovertemplate="Mode: %{x}<extra></extra>", mode="lines")
mean_line = go.Scatter(x=[mean]*100, y=np.linspace(0, yrange, 100),
                       name="Mean", 
                       hovertemplate="Mean: %{x}<extra></extra>", mode="lines")

fig = go.Figure()
fig.add_trace(bars)
fig.add_trace(mode_line)
fig.add_trace(mean_line)
fig.update_yaxes(range=[0, yrange])
fig.update_xaxes(range=[-0.5, 100.5])


graph = go.FigureWidget(fig)
graph.update_layout(
        autosize=True,
        margin=dict(
            l=20,
            r=20,
            b=20,
            t=100,
            pad=0),
        yaxis=dict(
            range=[0, yrange]),
)

def response(change):
    prior = np.ones(101) / 101 
    history_results = history.value
    posterior = apply_history_results(prior, history_results)
    mode = posterior.argmax()
    mean = np.average(np.arange(0, 101), weights=posterior)
    yrange = (np.ceil(posterior.max() * 100)+1) / 100
    with graph.batch_update():
        graph.data[0].y = posterior
        graph.data[1].x = [mode]*100
        graph.data[1].y = np.linspace(0, yrange, 100)
        graph.data[2].x = [mean]*100
        graph.data[2].y = np.linspace(0, yrange, 100)
        graph.layout["yaxis"].update(range=[0, yrange])

history.observe(response)

In [ ]:
widgets.VBox([history, graph])